In [ ]:
!pip install pandas>=2.0.0 numpy>=1.24.0 langchain>=1.0.0 langchain-core>=1.0.0 langchain-community>=0.4.0 langchain-text-splitters>=1.0.0 langchain-upstage>=0.7.0 faiss-cpu>=1.12.0 wikipedia-api nltk>=3.8.0 regex openpyxl>=3.1.0 tqdm>=4.65.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
transformers 4.57.2 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is incompatible.


In [ ]:
# 환경 설정 및 라이브러리 임포트
import os
import sys
import re
import time
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Optional, Dict, Tuple
from collections import Counter


# LangChain & Upstage
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader

# Wikipedia (지정된 API만 사용)
import wikipediaapi

In [ ]:
# Install gdown for downloading files from Google Drive if you plan to use it
!pip install gdown

### Download and Extract Vector Stores

**IMPORTANT**: Replace `YOUR_DRIVE_ID_FOR_EWHA` and `YOUR_DRIVE_ID_FOR_INTEGRATED` with the actual Google Drive file IDs of your zipped vector store directories. If your files are hosted elsewhere, please adjust the download commands accordingly (e.g., using `wget`).

In [ ]:
# Download EWHA vector store (example using gdown - replace with your file ID)
!gdown --id YOUR_DRIVE_ID_FOR_EWHA -O rag_ewha_vectorstore.zip
!unzip -o rag_ewha_vectorstore.zip -d .

# Download integrated vector store (example using gdown - replace with your file ID)
!gdown --id YOUR_DRIVE_ID_FOR_INTEGRATED -O rag_ewha_vectorstore_with_wiki.zip
!unzip -o rag_ewha_vectorstore_with_wiki.zip -d .

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=YOUR_DRIVE_ID_FOR_EWHA

but Gdown can't. Please check connections and permissions.
unzip:  cannot find or open rag_ewha_vectorstore.zip, rag_ewha_vectorstore.zip.zip or rag_ewha_vectorstore.zip.ZIP.
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file u

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# After running this, your Google Drive will be accessible at /content/drive/MyDrive/
# For example, if your folder is '4_NLP/4_NLP' inside MyDrive, the path would be:
# '/content/drive/MyDrive/4_NLP/4_NLP'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# API Key 설정
os.environ.setdefault("UPSTAGE_API_KEY", "up_g01lLro3nefoqy9IZGw3NK4ExjfIQ") # 본인의 키로 대체하세요
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY", "")
embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

# 작업 디렉토리 설정 (이 부분은 Colab VM의 기본 경로를 유지합니다)
CURRENT_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
# if Path.cwd() != CURRENT_DIR:
#       os.chdir(CURRENT_DIR) # 이 코드는 불필요하므로 주석 처리하거나 제거합니다.

print(f"✅ 작업 디렉토리: {CURRENT_DIR}")

# ==============================
# Load EWHA vector store
# ==============================
# EWHA 벡터스토어 경로를 Google Drive의 실제 경로로 지정합니다.
# 아래 경로는 예시이며, 사용자님의 실제 Google Drive 폴더 구조에 맞게 수정해야 합니다.
# 예: '/content/drive/MyDrive/MyProjectFolder/rag_ewha_vectorstore'
DRIVE_BASE_PATH = Path('/content/drive/MyDrive/4_NLP/4_NLP') # 이 경로를 사용자님의 실제 경로로 수정하세요!

EWHA_VECTOR_DB_PATH = DRIVE_BASE_PATH / "rag_ewha_vectorstore"
ewha_vector_store = None # Initialize to None

if EWHA_VECTOR_DB_PATH.exists() and (EWHA_VECTOR_DB_PATH / "index.faiss").exists():
    try:
        ewha_vector_store = FAISS.load_local(
            folder_path=str(EWHA_VECTOR_DB_PATH),
            embeddings=embeddings,
            allow_dangerous_deserialization=True
        )
        # EWHA retriever 생성
        ewha_retriever = ewha_vector_store.as_retriever(search_kwargs={"k": 4})
        print("📚 EWHA retriever ready")
    except RuntimeError as e:
        print(f"⚠️ EWHA 벡터스토어 로드 중 오류 발생: {e}. 파일이 손상되었거나 FAISS 버전이 일치하지 않을 수 있습니다.")
        ewha_retriever = None # Ensure it's not used if loading failed
else:
    print(f"⚠️ EWHA 벡터스토어 폴더 또는 'index.faiss' 파일이 '{EWHA_VECTOR_DB_PATH}' 경로에 없습니다. Google Drive 경로를 확인하거나 파일이 Drive에 있는지 확인하세요.")
    ewha_retriever = None # Ensure it's not used if loading failed

# ==============================
# Load integrated (EWHA + Wikipedia) vector store
# ==============================
INTEGRATED_VECTOR_DB_PATH = DRIVE_BASE_PATH / "rag_ewha_vectorstore_with_wiki"  # 저장된 경로명
integrated_vector_store = None

if INTEGRATED_VECTOR_DB_PATH.exists() and (INTEGRATED_VECTOR_DB_PATH / "index.faiss").exists():
    try:
        integrated_vector_store = FAISS.load_local(
            folder_path=str(INTEGRATED_VECTOR_DB_PATH),
            embeddings=embeddings,
            allow_dangerous_deserialization=True
        )
        print("🌍 Integrated vector store loaded (Wikipedia 포함)")
    except RuntimeError as e:
        print(f"⚠️ 통합 벡터스토어 로드 중 오류 발생: {e}. 파일이 손상되었거나 FAISS 버전이 일치하지 않을 수 있습니다.")
else:
    print(f"⚠️ 통합 벡터스토어 폴더 또는 'index.faiss' 파일이 '{INTEGRATED_VECTOR_DB_PATH}' 경로에 없습니다. Google Drive 경로를 확인하거나 파일이 Drive에 있는지 확인하세요.")

✅ 작업 디렉토리: /content
📚 EWHA retriever ready
🌍 Integrated vector store loaded (Wikipedia 포함)


After mounting your Drive, you would then need to modify cell `2456622c` to point to the correct paths within your mounted Google Drive, if that's where your `rag_ewha_vectorstore` and `rag_ewha_vectorstore_with_wiki` directories are located. For example, if they are directly inside `내드라이브/4_NLP/4_NLP`, your paths would look like this (adjusting for your exact folder structure):

In [ ]:
# ==================================================================
# 3. MMLU/일반 상식용 외부 검색 도구 (External Tools)""
# ==================================================================
import nltk

# NLTK 리소스 (한 번만 다운로드, 이후엔 캐시 사용)
try:
    nltk.download('punkt', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
except:
    pass

def extract_wiki_terms(question: str, max_terms: int = 3) -> List[str]:
    """
    질문에서 Wikipedia 검색용 후보 키워드를 추출한다.
    - NLTK 기반 고유명사(NNP), 길이가 긴 명사 우선
    - 도메인별 정교화된 키워드 매핑 적용
    - 불필요한 단어 필터링 (QUESTION, QUESTION26 등)
    """
    q = question.lower()

    # 불필요한 단어 필터링 리스트
    stop_words = {'question', 'questions', 'question26', 'question27', 'question28', 'question29',
                  'question30', 'following', 'information', 'refers', 'refers to', 'this', 'that',
                  'which', 'what', 'who', 'when', 'where', 'why', 'how', 'the', 'a', 'an',
                  'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
                  'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might',
                  'can', 'cannot', 'must', 'shall', 'all', 'some', 'any', 'none', 'both',
                  'each', 'every', 'other', 'another', 'such', 'same', 'different', 'various'}

    # 1️⃣ 도메인 기반 우선 처리 맵 (정밀 매핑)
    topic_map = [
        # 철학
        (["kohlberg"], ["Kohlberg's stages of moral development"]),
        (["moral development"], ["Kohlberg's stages of moral development"]),
        (["kant", "imperative"], ["Categorical imperative", "Immanuel Kant"]),
        (["utilitarian", "pleasure", "pain"], ["Utilitarianism", "Jeremy Bentham"]),
        (["positivism"], ["Legal positivism"]),

        # 형법 / 범죄
        (["robbery"], ["Robbery"]),
        (["larceny"], ["Larceny"]),
        (["burglary"], ["Burglary"]),
        (["stolen", "receiving"], ["Receiving stolen property"]),

        # Singer 영역
        (["singer", "affluence", "famine"], ["Famine, Affluence and Morality", "Peter Singer"]),
    ]

    # 1️⃣ 도메인 기반 맵핑 그대로
    for triggers, wiki_titles in topic_map:
        if any(t in q for t in triggers):
            return wiki_titles[:max_terms]

    # 2️⃣ NLTK noun 기반 fallback
    try:
        tokens = nltk.word_tokenize(question)
        tagged = nltk.pos_tag(tokens)
    except Exception:
        tokens = re.findall(r"[A-Za-z][A-Za-z\-]{2,}", question)
        filtered_tokens = [t for t in tokens if t.lower() not in stop_words]
        return filtered_tokens[:max_terms]

    candidates = [
        word for word, pos in tagged
        if pos.startswith("NNP") or (pos.startswith("NN") and len(word) >= 6)
    ]

    cleaned = []
    for w in candidates:
        w = re.sub(r"[^A-Za-z\-]", "", w)
        if len(w) >= 3 and w.lower() not in stop_words:
            cleaned.append(w)

    unique = []
    seen = set()
    for w in cleaned:
        wl = w.lower()
        if wl not in seen:
            seen.add(wl)
            unique.append(w)

    return unique[:max_terms]

def retrieve_wikipedia_docs(query: str, lang: str = 'en') -> List[Document]:
    """
    Wikipedia 검색 및 문서 반환 (동적)
    개선: NLTK 기반 키워드 추출 후 검색
    """
    docs = []

    # 1. NLTK 기반 키워드 추출
    try:
        wiki_keywords = extract_wiki_terms(query, max_terms=3)
        # 원본 query도 키워드에 추가
        search_queries = [query] + wiki_keywords
        if wiki_keywords:
            print(f"   🔍 NLTK 검색: '{query}' → 키워드 {wiki_keywords}")
    except:
        search_queries = [query]

    # 2. 각 키워드로 Wikipedia 검색
    seen_titles = set()
    for search_query in search_queries[:3]:  # 최대 3개 키워드만 사용
        if not search_query or search_query in seen_titles:
            continue

        try:
            # wikipedia-api 사용
            wiki_wiki = wikipediaapi.Wikipedia(
                user_agent='MyNLPProject/1.0',
                language=lang,
                extract_format=wikipediaapi.ExtractFormat.WIKI
            )
            page = wiki_wiki.page(search_query)

            if page.exists() and page.title not in seen_titles:
                seen_titles.add(page.title)
                text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=80)
                chunks = text_splitter.split_text(page.text)
                for i, chunk in enumerate(chunks[:5]): # 상위 5개 청크만 사용
                    docs.append(Document(
                        page_content=chunk,
                        metadata={
                            "source": "Wikipedia",
                            "title": page.title,
                            "type": "external",
                            "priority": "medium"
                        }
                    ))

                if docs:
                    print(f"   🔍 동적 Wikipedia 검색: '{search_query}' → {len([d for d in docs if d.metadata.get('title') == page.title])}개 청크")

                # 최대 2개 페이지만 검색 (성능 향상)
                if len(seen_titles) >= 2:
                    break
        except Exception as e:
            pass # 에러 무시

    return docs

def extract_keywords_enhanced(question: str, choices: List[str] = None) -> List[str]:
    keywords = []

    # 핵심 개념은 extract_wiki_terms로 단일화
    try:
        keywords.extend(extract_wiki_terms(question, max_terms=3))
    except:
        pass

    # 선택지 기반 확장만
    if choices:
        for choice in choices:
            choice_keywords = re.findall(r'\b[a-zA-Z]{3,}\b', choice)
            keywords.extend(choice_keywords)

    return list(dict.fromkeys(keywords))[:5]  # 중복 제거 + 순서 유지


In [ ]:
# ==================================================================
# 4. Router & Prompts (도메인 분리 전략)
# ==================================================================

# LLM 모델 정의 (전역 변수로 선언)
llm_model = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2", temperature=0.1)

# 라우팅 로직 (개선: LLM 기반 질문 분류 + 한글 체크 하이브리드)
def classify_question(question: str) -> bool:
    """
    LLM을 사용하여 질문이 이화여대 학칙 관련인지 분류
    Returns: True if EWHA-related, False if general/MMLU
    """
    # 먼저 간단한 한글 체크로 빠르게 필터링
    has_korean = any('\uAC00' <= c <= '\uD7A3' for c in question)
    if has_korean:
        return True  # 한글이 있으면 대부분 이화여대 문제

    # 영어 문제는 기본적으로 MMLU로 분류 (성능 향상 및 에러 방지)
    # 단, 명확히 이화여대 관련 키워드가 있으면 LLM으로 재확인
    ewha_keywords = ['ewha', 'euhwa', '이화여자대학교', '이화여대', 'ewha university']
    question_lower = question.lower()

    # 명확히 이화여대 관련 키워드가 있으면 EWHA로 분류
    if any(keyword in question_lower for keyword in ewha_keywords):
        return True

    # 영어 문제는 기본적으로 MMLU로 분류 (LLM 호출 생략)
    # 필요시에만 LLM 호출 (선택적)
    use_llm_classification = False  # 성능 향상을 위해 기본값은 False

    if use_llm_classification:
        # 영어 문제인 경우 선택적으로 LLM으로 분류 (더 정확한 판단)
        classification_prompt_template = PromptTemplate.from_template(
            """
다음 질문이 이화여자대학교 학칙, 학사 행정, 입학/졸업 규정, 전공/부전공, 학점, 수업 연한 등
이화여대의 내부 행정 규정에 관한 질문인지 판단하세요.

질문: {question}

위 질문이 이화여자대학교의 학칙이나 학사 행정 규정에 관한 것이면 "YES",
일반적인 상식, 법률, 심리학, 역사, 철학, 경제 등 일반 지식 문제이면 "NO"로만 답변하세요.

답변 (YES 또는 NO만):
"""
        )

        try:
            # 타임아웃 설정을 위한 LLM (더 빠른 응답)
            quick_llm = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2", temperature=0.1)
            classification_chain = classification_prompt_template | quick_llm

            # 질문이 너무 길면 앞부분만 사용
            question_truncated = question[:500] if len(question) > 500 else question

            response = classification_chain.invoke({"question": question_truncated})
            answer = response.content.strip().upper()

            # YES가 포함되면 EWHA 관련
            return "YES" in answer
        except Exception as e:
            # 에러 발생 시 기본값: False (MMLU로 분류)
            print(f"⚠️ 질문 분류 에러: {type(e).__name__}: {str(e)[:100]}, MMLU로 분류")
            return False
    else:
        # 기본적으로 영어 문제는 MMLU로 분류
        return False

def classify_domain(question: str) -> str:
    """
    질문이 'Ewha' 관련인지 'MMLU' 인지 분류 (개선: LLM 기반)
    - 한글이 포함되어 있으면 이화문제 (EWHA)
    - 영어 문제는 LLM으로 분류
    """
    is_ewha = classify_question(question)
    return "EWHA" if is_ewha else "MMLU"

# 템플릿 1: 이화여대 학사 행정 전문가 (상세 버전)
ewha_prompt = PromptTemplate.from_template(
    """
    당신은 이화여자대학교 학사 행정 전문가입니다.
    제공된 [이화여대 학칙/규정]을 꼼꼼히 분석하여 질문에 가장 정확한 답변을 선택하세요.

    [지침]
    1. 질문의 핵심 의도를 파악하세요.
    2. [이화여대 학칙/규정]에서 관련 조항을 찾으세요.
    3. 각 선택지(A, B, C, D...)를 규정과 비교하여 검증하세요.
    4. 부정형 질문("하지 않는", "아닌")에 특히 주의하세요.
    5. 규정에 명시되지 않은 내용은 "정보 없음"으로 판단하세요.
    6. 최종적으로 가장 확실한 정답 하나를 선택하세요.

    질문: {question}

    [이화여대 학칙/규정]:
    {context}

    생각하는 과정:
    1) 관련 규정 확인: (관련된 조항 번호와 내용)
    2) 선택지 검증:
       - A: (규정 위반 여부 / 일치 여부)
       - B: ...
    3) 결론 도출: (정답 선택 이유)

    최종 답변 (정답 형식: "정답: (번호)"):
    """
)

# 템플릿 2: 해당 분야의 교수 (MMLU용 - 개선: 참고 지식 직접 언급 강조)
mmlu_prompt = PromptTemplate.from_template(
    """
    당신은 해당 분야의 세계적인 석학입니다.
    제공된 [참고 지식]을 바탕으로 질문에 가장 정확한 답변을 선택하세요.

    [지침]
    1. 질문의 핵심 개념을 파악하세요.
    2. [참고 지식]에서 관련 정보를 찾아 각 선택지를 검증하세요.
    3. **모든 선택지(A~J)를 빠짐없이 검토**하세요. 특히 I, J 선택지도 반드시 확인하세요.
    4. **중요**: 참고 지식에서 직접적으로 언급되거나 명시적으로 지지되는 선택지를 우선하세요.
      - 참고 지식에 구체적으로 나와 있는 선택지가 정답일 가능성이 높습니다.
      - 추론보다는 참고 지식의 직접적 언급을 우선하세요.
    5. 법률 용어, 단계 번호, 숫자/백분율 등은 정확히 확인하세요.
    6. 여러 선택지가 부분적으로 타당해 보여도, 참고 지식에서 가장 명확하게 지지하는 하나를 선택하세요.

    질문: {question}

    [참고 지식]:
    {context}

    생각하는 과정:
    1) 핵심 개념: (질문의 주요 키워드)
    2) 참고 지식 분석: (관련 정보 요약, 특히 직접적으로 언급된 내용)
    3) 선택지 검토 (모든 선택지 A~J를 순차적으로):
       - 각 선택지를 참고 지식과 비교
       - 참고 지식에서 이 선택지를 직접 언급하는 부분이 있는지 확인
       - 특히 I, J 선택지도 반드시 검토 (마지막 선택지가 정답일 수 있음)
    4) 최종 답 선택:
       - 참고 지식에서 가장 직접적으로 지지하는 선택지
       - 참고 지식의 구체적 인용 포함

    최종 답변 (정답 형식: "정답: (알파벳)"):
    """
)


In [ ]:
# ==================================================================
# 5. Enhanced Retrieval & Unified Solver (개선: 재검색 로직 추가)
# ==================================================================

def extract_year_from_question(question: str) -> str:
    """질문에서 연도(4자리 숫자) 추출"""
    matches = re.findall(r'\b(20\d{2})', question)
    if matches:
        return matches[0]
    return ""

def extract_keywords_from_question(question: str) -> List[str]:
    """질문에서 검색 키워드 추출 (재검색용)"""
    keywords = []

    # 질문에서 주요 명사/키워드 추출
    # 영어 단어 추출
    english_words = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b', question)
    keywords.extend([w.lower() for w in english_words if len(w) > 3])

    # 한국어 키워드 추출 (2글자 이상)
    korean_words = re.findall(r'[가-힣]{2,}', question)
    keywords.extend(korean_words)

    # 선택지에서 키워드 추출
    choice_pattern = r'\([A-Z]\)\s*([^\(\)]+?)(?=\s*\([A-Z]\)|$)'
    choices = re.findall(choice_pattern, question)
    for choice in choices:
        # 선택지에서 중요한 단어 추출
        words = re.findall(r'\b[A-Z][a-z]+\b|[가-힣]{2,}', choice)
        keywords.extend([w.lower() if w.isalpha() else w for w in words if len(w) > 2])

    # 중복 제거 후 반환
    return list(set(keywords))[:5]  # 상위 5개만


def enhanced_retrieval(question: str, retriever, is_ewha_related: bool, max_docs: int = 10) -> List[Document]:
    """
    향상된 검색 (재검색 로직 포함)
    - 첫 검색 결과가 부족하면 키워드 기반 재검색 수행
    """
    # 첫 번째 검색
    source_docs = retriever.invoke(question)
    existing_sources = {doc.metadata.get('source', '') + str(doc.metadata.get('article_index', '')) for doc in source_docs}

    # 검색 결과가 부족하거나 관련성이 낮으면 재검색
    if len(source_docs) < 5 or (is_ewha_related and len(source_docs) < 3):
        # 키워드 기반 재검색
        keywords = extract_keywords_from_question(question)
        if keywords:
            # 가장 중요한 키워드로 재검색
            for keyword in keywords[:3]:  # 상위 3개 키워드만 사용
                try:
                    additional_docs = retriever.invoke(keyword)
                    for doc in additional_docs:
                        doc_id = doc.metadata.get('source', '') + str(doc.metadata.get('article_index', ''))
                        if doc_id not in existing_sources:
                            source_docs.append(doc)
                            existing_sources.add(doc_id)
                except:
                    continue
                if len(source_docs) >= max_docs:
                    break

    return source_docs[:max_docs]  # 최대 개수 제한

def solve_question(question: str, llm_model, temperature: float = 0.1) -> str:
    """
    질문 해결 (개선: 선택지별 개별 검증 추가)
    """
    domain = classify_domain(question)
    context_text = ""


    # --- [경로 A] 이화여대 학칙 문제 ---
    if domain == "EWHA":
        year = extract_year_from_question(question)
        search_query = question
        if year:
            search_query = f"{year}학년도 {question}" # 연도 강조
        # 1. 향상된 검색 (재검색 로직 포함)
        docs = enhanced_retrieval(search_query, ewha_retriever, is_ewha_related=True, max_docs=10)

        # 2. 검색 결과 필터링 (유사도 점수가 있으면 필터링)
        filtered_docs = []
        priority_docs = []
        other_docs = []

        for doc in docs:
            # 연도가 질문에 있는데 문서에 없으면 우선순위 낮춤
            if year:
                meta_year = str(doc.metadata.get('year', ''))
                doc_content = doc.page_content
                if year in meta_year or year in doc_content:
                    priority_docs.append(doc)
                else:
                    other_docs.append(doc)
            else:
                priority_docs.append(doc)

        filtered_docs = priority_docs + other_docs
        if not filtered_docs:
            filtered_docs = docs[:8]  # 최대 8개 사용

        # 3. 문맥 생성 (문서 번호 추가로 참조 용이)
        context_parts = []
        for i, doc in enumerate(filtered_docs[:8], 1):  # 최대 8개 사용
            doc_text = doc.page_content
            source_info = doc.metadata.get('source', 'Unknown')
            context_parts.append(f"[문서 {i}] (출처: {source_info})\n{doc_text}")

        context_text = "\n\n".join(context_parts) or ""

        # 4. LLM 호출 (Strict Prompt)
        chain = ewha_prompt | llm_model

    # --- [경로 B] MMLU / 일반 상식 문제 ---
    else:
        # 1. 키워드 추출 (NLTK 기반)
        # 선택지 추출 (괄호 안 알파벳 패턴)
        choices = re.findall(r'\([A-Z]\)\s*(.*?)(?=\(|$)', question)
        keywords = extract_keywords_enhanced(question, choices)

        # 2. 하이브리드 검색 전략 (개선: 더 많은 문서 검색 + 선택지 키워드 재검색)
        # 2-1. 사전 벡터화된 통합 벡터 스토어에서 검색 (개선: k=10으로 증가)
        docs = []
        if 'integrated_vector_store' in globals() and integrated_vector_store is not None:
            try:
                mmlu_retriever = integrated_vector_store.as_retriever(search_kwargs={"k": 10})
                vector_docs = mmlu_retriever.invoke(question)
                # Wikipedia 문서만 필터링 (이화여대 문서 제외)
                wiki_from_vector = [d for d in vector_docs if d.metadata.get('type') == 'wikipedia']
                docs.extend(wiki_from_vector)
                if wiki_from_vector:
                    print(f"   📚 사전 벡터화 Wikipedia 문서에서 {len(wiki_from_vector)}개 검색")
            except Exception as e:
                print(f"   ⚠️ 통합 벡터 스토어 검색 실패: {e}")

        # 2-2. 동적 Wikipedia 검색 (추가)
        # 질문 자체로 검색
        dynamic_wiki_docs = retrieve_wikipedia_docs(question)
        docs.extend(dynamic_wiki_docs)

        # 키워드로 추가 검색
        for kw in keywords[:2]:
            dynamic_docs = retrieve_wikipedia_docs(kw)
            docs.extend(dynamic_docs)
            # 'definition' 추가 검색 제거 (불필요한 검색 방지)

        # 2-3. 선택지 키워드 기반 재검색 (오답 개선 핵심)
        # 선택지에서 핵심 키워드 추출하여 재검색
        if choices:
            choice_keywords = []
            for choice in choices:
                # 선택지에서 중요한 법률/철학/심리학/역사 용어 추출
                important_terms = re.findall(r'\b(?:strict liability|product liability|assault|battery|negligence|vicarious|admissible|evidence|character evidence|propensity|paternity|natural law|unnatural|emergency|obedience|milgram|dershowitz|aquinas|kohlberg|stage|phase|lithic|wedge-shaped|micro-blade|arctic|denali|nenana|tang|nomadic|frontier|military|power|projection)\b', choice.lower())
                choice_keywords.extend(important_terms)

                # 선택지에서 대문자로 시작하는 고유명사/개념 추출
                proper_nouns = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b', choice)
                choice_keywords.extend([w.lower() for w in proper_nouns if len(w) > 4])

            # 중복 제거 후 재검색
            unique_choice_keywords = list(set(choice_keywords))[:5]  # 상위 5개만
            for kw in unique_choice_keywords:
                additional_docs = retrieve_wikipedia_docs(kw)
                docs.extend(additional_docs)
                # "definition" 또는 "law" 자동 추가 검색 제거 (불필요한 검색 방지)
                # 실제 개념/용어는 Wikipedia에서 직접 검색되는 것이 더 효과적

        # 3. 문맥 생성 (중복 제거 + 재랭킹)
        seen = set()
        unique_docs = []
        for d in docs:
            # 중복 체크 (내용 기반)
            content_hash = hash(d.page_content[:100])  # 앞 100자로 중복 체크
            if content_hash not in seen:
                seen.add(content_hash)
                unique_docs.append(d)

        # 재랭킹: 질문과의 관련성 기반 점수 계산
        if unique_docs:
            def score_doc(d):
                score = 0
                content_lower = d.page_content.lower()
                question_lower = question.lower()

                # 1. 문서 타입 점수
                if d.metadata.get('type') == 'wikipedia':
                    score += 5
                if d.metadata.get('priority') == 'high':
                    score += 4

                # 2. 키워드 매칭 점수 (질문의 키워드가 문서에 얼마나 많이 나타나는지)
                question_keywords = set(re.findall(r'\b[a-z]{4,}\b', question_lower))
                doc_keywords = set(re.findall(r'\b[a-z]{4,}\b', content_lower))
                common_keywords = question_keywords & doc_keywords
                score += len(common_keywords) * 2

                # 3. 정의/개념 설명 포함 여부
                if any(word in content_lower for word in ['definition', 'refers to', 'means', 'is defined', 'concept']):
                    score += 3

                # 4. 선택지 키워드 매칭 (선택지의 핵심 단어가 문서에 있는지)
                if choices:
                    for choice in choices[:5]:  # 상위 5개 선택지 확인
                        choice_keywords = set(re.findall(r'\b[a-z]{4,}\b', choice.lower()))
                        if choice_keywords & doc_keywords:
                            score += 1

                # 5. 법률/철학/심리학/역사 특정 용어 매칭
                if any(term in content_lower for term in ['evidence', 'negligence', 'liability', 'tort', 'law']):
                    if any(term in question_lower for term in ['evidence', 'negligence', 'liability', 'tort', 'law']):
                        score += 2

                if any(term in content_lower for term in ['aquinas', 'dershowitz', 'natural law', 'ethics']):
                    if any(term in question_lower for term in ['aquinas', 'dershowitz', 'natural law', 'ethics']):
                        score += 2

                if any(term in content_lower for term in ['milgram', 'obedience', 'experiment']):
                    if any(term in question_lower for term in ['milgram', 'obedience', 'experiment']):
                        score += 2

                if any(term in content_lower for term in ['tang', 'dynasty', 'nomadic', 'frontier', 'military']):
                    if any(term in question_lower for term in ['tang', 'nomadic', 'frontier', 'military']):
                        score += 2

                return score

            # 점수 기반 재랭킹
            unique_docs.sort(key=lambda x: score_doc(x), reverse=True)

            # 상위 점수 문서만 선택 (품질 기반 필터링)
            scored_docs = [(score_doc(d), d) for d in unique_docs]
            scored_docs.sort(key=lambda x: x[0], reverse=True)

            # 상위 점수 문서 선택 (최소 점수 3 이상 또는 상위 8개)
            filtered_scored = [d for score, d in scored_docs if score >= 3][:8]
            if not filtered_scored:
                # 점수가 낮아도 상위 6개는 사용
                filtered_scored = [d for _, d in scored_docs[:6]]

            unique_docs = filtered_scored

        # 검색 결과를 6개로 제한 (너무 많으면 노이즈 증가)
        context_text = "\n\n".join([d.page_content for d in unique_docs[:6]])

        # 4. 선택지별 핵심 키워드 재검색 (오답 개선: Q3, Q4, Q7, Q19, Q23)
        # 선택지에서 중요한 법률/철학 용어를 추출하여 추가 검색
        if choices:
            choice_keywords = []
            for choice in choices:
                # 법률 용어 추출
                legal_terms = re.findall(r'\b(?:negligence|vicarious liability|assault|battery|trespass|demonstrative evidence|relevance|admissibility|paternity|character evidence|propensity evidence)\b', choice.lower())
                choice_keywords.extend(legal_terms)

                # 철학 용어 추출
                philosophy_terms = re.findall(r'\b(?:emergency ethics|ticking time bomb|natural law|unnatural acts|procreation|reproductive purpose)\b', choice.lower())
                choice_keywords.extend(philosophy_terms)

                # 마지막 선택지(I, J)의 키워드도 추출
                if len(choices) >= 2 and (choice == choices[-1] or choice == choices[-2]):
                    # I, J 선택지의 핵심 단어 추출
                    important_words = re.findall(r'\b[a-z]{5,}\b', choice.lower())
                    choice_keywords.extend(important_words[:3])  # 상위 3개만

            # 중복 제거 후 추가 검색
            unique_choice_keywords = list(set(choice_keywords))[:4]  # 상위 4개만
            additional_docs = []
            for kw in unique_choice_keywords:
                try:
                    extra_docs = retrieve_wikipedia_docs(kw)
                    additional_docs.extend(extra_docs)
                except:
                    pass

            # 추가 문서를 컨텍스트에 포함 (최대 2개만)
            if additional_docs:
                seen_additional = set()
                for d in additional_docs[:2]:
                    content_hash = hash(d.page_content[:100])
                    if content_hash not in seen_additional:
                        seen_additional.add(content_hash)
                        context_text += "\n\n" + d.page_content

        # 5. LLM 호출 (Academic Prompt)
        chain = mmlu_prompt | llm_model

    # 최종 답변 생성
    try:
        response = chain.invoke({"question": question, "context": context_text})
        return response.content
    except Exception as e:
        print(f"   ⚠️ LLM 호출 에러: {e}")
        return "Unknown"


In [ ]:
# ==================================================================
# 6. Self-Consistency (Majority Voting) Wrapper
# ==================================================================

def extract_answer_from_response(response: str) -> str:
    """
    응답에서 정답 추출 (개선 버전 - RAG_다수결.ipynb 방식)
    """
    if not response or not response.strip():
        return "Unknown"

    # 유효한 선택지 (A부터 J까지)
    valid_choices = set("ABCDEFGHIJ")

    # 패턴 1: "정답: (A)" 형식 우선 검색
    patterns = [
        r'정답[:\s]*\(([A-J])\)',
        r'정답[:\s]*([A-J])\b',
        r'\(([A-J])\)\s*[가-힣]*정답',
        r'최종\s*답변[:\s]*\(([A-J])\)',
        r'최종\s*답변[:\s]*([A-J])\b',
    ]

    for pattern in patterns:
        matches = list(re.finditer(pattern, response, re.IGNORECASE))
        if matches:
            for match in reversed(matches):  # 뒤에서부터 (최종 답변일 가능성)
                answer = match.group(1).upper()
                if answer in valid_choices:
                    return answer

    # 패턴 2: "(A)" 형식 단독 추출 (문맥에서 명확한 경우)
    matches = list(re.finditer(r'\(([A-J])\)', response))
    if matches:
        # 뒤에서부터 검색 (최종 답변일 가능성)
        for match in reversed(matches):
            answer = match.group(1).upper()
            if answer in valid_choices:
                # 주변 텍스트 확인 (명확한 답변인지)
                start = max(0, match.start() - 10)
                end = min(len(response), match.end() + 10)
                context = response[start:end].lower()
                if any(word in context for word in ['정답', '답', '선택', 'answer']):
                    return answer

    # 패턴 3: 단독 문자 (마지막 선택)
    matches = list(re.finditer(r'\b([A-J])\b', response, re.IGNORECASE))
    if matches:
        # 뒤에서부터 검색
        for match in reversed(matches):
            answer = match.group(1).upper()
            if answer in valid_choices:
                # 주변 텍스트 확인
                start = max(0, match.start() - 10)
                end = min(len(response), match.end() + 10)
                context = response[start:end].lower()
                if any(word in context for word in ['정답', '답', '선택', 'answer', '결론']):
                    return answer

    return "Unknown"

def solve_with_consistency(question: str, num_samples: int = 3, allow_tie_retry: bool = True):
    """
    Self-Consistency 기반 다수결 + 동률 처리
    - 샘플 수 3으로 유지, temperature 다양화 (0.3, 0.5, 0.7)
    - 1:1:1 동률 발생 시 추가 추론 2회 수행 (동률 해소될 때까지)
    - 다른 동률(예: 2:2:1) 발생 시 재시도 최대 3회 수행 (동률 해소될 때까지)
    - 재시도 후에도 동률이 남으면 (예: 2:2:2) 계속 추가 추론 진행
    """
    # Temperature 다양화: 다양한 관점에서 답변 생성
    temperatures = [0.3, 0.5, 0.7]

    votes = []
    raw_responses = []

    print(f"🤔 Thinking (Consistency k={num_samples})...")

    # 1차 투표 (다양한 temperature로)
    for i in range(num_samples):
        temp = temperatures[i % len(temperatures)]
        voter_llm = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2", temperature=temp)
        resp = solve_question(question, voter_llm)
        raw_responses.append(resp)
        votes.append(extract_answer_from_response(resp))

    counter = Counter(votes)
    print("   🗳️ 1차 투표:", dict(counter))

    # -----------------------------
    # 🔥 동률 처리 로직 (개선)
    # -----------------------------
    if allow_tie_retry and len(counter) > 1:
        counts_only = list(counter.values())
        max_count = max(counts_only)

        # 1:1:1 동률 체크 (3개 선택지가 각각 1표씩)
        is_1_1_1_tie = (len(counter) == 3 and all(cnt == 1 for cnt in counts_only))

        if is_1_1_1_tie:
            # 1:1:1 동률인 경우 2번 더 추론 진행
            print("⚠️ 1:1:1 동률 발생 → 추가 추론 2회 수행")
            retry_temperatures = [0.2, 0.15]  # 더 낮은 temperature로 확실한 답변 유도

            for retry_num, retry_temp in enumerate(retry_temperatures, 1):
                retry_llm = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2", temperature=retry_temp)

                retry_resp = solve_question(
                    question + "\n\nThink step by step and decide the SINGLE best answer. Pay special attention to ALL choices including I and J. Choose the answer that is MOST DIRECTLY supported by the reference knowledge provided.",
                    retry_llm
                )
                retry_answer = extract_answer_from_response(retry_resp)

                if retry_answer != "Unknown":
                    raw_responses.append(retry_resp)
                    votes.append(retry_answer)
                    counter = Counter(votes)
                    print(f"   🔁 추가 추론 {retry_num}회 결과 반영:", dict(counter))

                    # 동률이 해소되면 중단
                    counts_after = list(counter.values())
                    max_count_after = max(counts_after)
                    top_counts_after = [cnt for cnt in counts_after if cnt == max_count_after]
                    if len(top_counts_after) == 1:
                        print(f"   ✅ 동률 해소됨 (추가 추론 {retry_num}회 후)")
                        break

        else:
            # 다른 동률 (예: 2:2:1, 2:1:1:1 등)인 경우 재시도 후에도 동률이 남으면 계속 추론
            top_counts = [cnt for cnt in counts_only if cnt == max_count]
            if len(top_counts) > 1:  # 여러 선택지가 동일한 최빈값을 가진 경우
                print(f"⚠️ 동률 발생 ({dict(counter)}) → 재시도 수행 (동률 해소될 때까지)")
                retry_temperatures = [0.2, 0.15, 0.1]  # 동률이 해소될 때까지 최대 3회 재시도

                for retry_num, retry_temp in enumerate(retry_temperatures, 1):
                    retry_llm = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2", temperature=retry_temp)

                    retry_resp = solve_question(
                        question + "\n\nThink step by step and decide the SINGLE best answer. Pay special attention to ALL choices including I and J. Choose the answer that is MOST DIRECTLY supported by the reference knowledge provided.",
                        retry_llm
                    )
                    retry_answer = extract_answer_from_response(retry_resp)

                    if retry_answer != "Unknown":
                        raw_responses.append(retry_resp)
                        votes.append(retry_answer)
                        counter = Counter(votes)
                        print(f"   🔁 재시도 {retry_num}회 결과 반영:", dict(counter))

                        # 동률이 해소되었는지 확인
                        counts_after = list(counter.values())
                        max_count_after = max(counts_after)
                        top_counts_after = [cnt for cnt in counts_after if cnt == max_count_after]

                        if len(top_counts_after) == 1:
                            print(f"   ✅ 동률 해소됨 (재시도 {retry_num}회 후)")
                            break
                        elif retry_num < len(retry_temperatures):
                            print(f"   ⚠️ 동률 지속 ({dict(counter)}) → 추가 재시도 진행")

    # 최종 결정
    final = counter.most_common(1)[0][0]

    # UNKNOWN 처리
    if final == "Unknown" and len(counter) > 1:
        for ans, cnt in counter.most_common():
            if ans != "Unknown":
                final = ans
                break

    # 가장 좋은 설명 선택 (가장 긴 설명 선택 - 더 상세한 분석)
    best_explanation = ""
    max_length = 0
    for resp, vote in zip(raw_responses, votes):
        if vote == final and len(resp) > max_length:
            max_length = len(resp)
            best_explanation = resp

    print(f"   ✅ Final decision: {final}")
    return final, best_explanation

In [ ]:
# ==================================================================
# 7. Main Execution Loop
# ==================================================================

# NOTE: This path MUST be updated to your actual testset.csv location on Google Drive.
# Example: Path('/content/drive/MyDrive/YourFolderName/testset.csv')
TESTSET_PATH = Path('/content/drive/MyDrive/4_NLP/4_NLP/testset.csv') # <--- 이 경로를 사용자님의 실제 testset.csv 경로로 수정하세요!

# Check if testset.csv exists
if not TESTSET_PATH.exists():
    print(f"❌ Error: Testset not found at {TESTSET_PATH}")
    print("Please ensure the path is correct and the file exists on your Google Drive.")
    print("Initializing an empty DataFrame for now. Please provide the testset to continue processing.")
    df = pd.DataFrame() # Initialize empty DataFrame to prevent NameError later
    # No further processing will occur without the testset.csv
else:
    df = pd.read_csv(TESTSET_PATH)

    print(f"📄 Loaded testset: {len(df)} questions")

    results = []
    all_gold = []
    all_pred_raw = []

    for idx, row in df.iterrows():
        q_num = idx + 1
        question = row['prompts']
        gold_answer = row.get('answers', '')

        print(f"\n[Q{q_num}] Processing...")
        domain = classify_domain(question)
        print(f"   🏷️ Domain: {domain}")

        final_pred, explanation = solve_with_consistency(question, num_samples=3)

        all_gold.append(gold_answer)
        all_pred_raw.append(final_pred)

        # 제출용 CSV에는 문제와 예측 답변만 저장!!
        results.append({
            "question": question,
            "your_answer": final_pred
        })

    # ------------------------------
    # CSV 구성
    # ------------------------------
    res_df = pd.DataFrame(results)

    # 정답 문자만 추출
    def extract_pred_answer(pred_str):
        if pd.isna(pred_str) or not pred_str or str(pred_str).strip() == "Unknown":
            return None
        pred_str = str(pred_str).strip()
        match = re.search(r"\(([A-Z])\)", pred_str) or re.search(r"([A-Z])\b", pred_str)
        if match:
            return match.group(1).upper()
        return pred_str.strip().upper()

    # your_answer 정제
    res_df["your_answer"] = res_df["your_answer"].apply(extract_pred_answer)

    # 최종 제출 파일명: 조번호 기반 (예: 4_final.csv)
    res_df.to_csv("4_final.csv", index=False, encoding="utf-8-sig")

    print("🎉 4_final.csv 생성 완료! (question + your_answer ONLY)")

📄 Loaded testset: 50 questions

[Q1] Processing...
   🏷️ Domain: EWHA
🤔 Thinking (Consistency k=3)...
   🗳️ 1차 투표: {'D': 3}
   ✅ Final decision: D

[Q2] Processing...
   🏷️ Domain: EWHA
🤔 Thinking (Consistency k=3)...
   🗳️ 1차 투표: {'A': 3}
   ✅ Final decision: A

[Q3] Processing...
   🏷️ Domain: EWHA
🤔 Thinking (Consistency k=3)...
   🗳️ 1차 투표: {'C': 3}
   ✅ Final decision: C

[Q4] Processing...
   🏷️ Domain: EWHA
🤔 Thinking (Consistency k=3)...
   🗳️ 1차 투표: {'D': 3}
   ✅ Final decision: D

[Q5] Processing...
   🏷️ Domain: EWHA
🤔 Thinking (Consistency k=3)...
   🗳️ 1차 투표: {'C': 3}
   ✅ Final decision: C

[Q6] Processing...
   🏷️ Domain: EWHA
🤔 Thinking (Consistency k=3)...
   🗳️ 1차 투표: {'B': 3}
   ✅ Final decision: B

[Q7] Processing...
   🏷️ Domain: EWHA
🤔 Thinking (Consistency k=3)...
   🗳️ 1차 투표: {'D': 3}
   ✅ Final decision: D

[Q8] Processing...
   🏷️ Domain: EWHA
🤔 Thinking (Consistency k=3)...
   🗳️ 1차 투표: {'C': 2, 'B': 1}
   ✅ Final decision: C

[Q9] Processing...
   🏷️ Domain:

In [ ]:
cnt = 0

for i, (gold, pred_raw) in enumerate(zip(all_gold[:50], all_pred_raw[:50]), 1):
    pred = extract_pred_answer(pred_raw)

    print("-" * 10, f"Q{i}", "-" * 10)
    print(f"generated: {pred}, gold: {gold}")

    if pred and pred in gold:
        cnt += 1

acc = (cnt / 50) * 100
print("\n=======================")
print(f"Final Accuracy: {acc:.1f}%")
print("=======================\n")

---------- Q1 ----------
generated: D, gold: (D)
---------- Q2 ----------
generated: A, gold: (A)
---------- Q3 ----------
generated: C, gold: (C)
---------- Q4 ----------
generated: D, gold: (D)
---------- Q5 ----------
generated: C, gold: (C)
---------- Q6 ----------
generated: B, gold: (B)
---------- Q7 ----------
generated: D, gold: (D)
---------- Q8 ----------
generated: C, gold: (C)
---------- Q9 ----------
generated: C, gold: (C)
---------- Q10 ----------
generated: B, gold: (B)
---------- Q11 ----------
generated: B, gold: (B)
---------- Q12 ----------
generated: B, gold: (B)
---------- Q13 ----------
generated: C, gold: (C)
---------- Q14 ----------
generated: C, gold: (C)
---------- Q15 ----------
generated: A, gold: (A)
---------- Q16 ----------
generated: B, gold: (B)
---------- Q17 ----------
generated: B, gold: (B)
---------- Q18 ----------
generated: D, gold: (D)
---------- Q19 ----------
generated: D, gold: (D)
---------- Q20 ----------
generated: D, gold: (D)
---------

In [21]:
from google.colab import files
files.download("4_final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>